In [ ]:
!pip install mlflow fastapi nest-asyncio pyngrok scikit-learn pandas

In [ ]:
from google.colab import files
files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d adityayadav01/customer-churn-analysis-dataset
!unzip customer-churn-analysis-dataset.zip


Saving customer-churn-analysis-dataset.zip to customer-churn-analysis-dataset.zip
cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 10, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/kaggle/cli.py", line 68, in main
    out = args.func(**command_args)
          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/kaggle/api/kaggle_api_extended.py", line 1741, in dataset_download_cli
    with self.build_kaggle_client() as kaggle:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/kaggle/api/kaggle_api_extended.py", line 688, in build_kaggle_client
    username=self.config_values['username'],
             ~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^
KeyError: 'username'
Archive:  customer-churn-analysis-dataset.zip
replace Customer Churn Analy

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("Customer Churn Analysis.csv")
print(df.shape)
df.head()

(1000, 11)


,Unnamed: 0,CustomerID,Gender,Age,Tenure,PhoneService,InternetService,Contract,MonthlyCharges,TotalCharges,Churn
0,0,CUST_0001,Male,71,69,Yes,Fiber optic,Month-to-month,45.08,3110.52,No
1,1,CUST_0002,Female,80,71,No,DSL,Month-to-month,80.37,5706.27,Yes
2,2,CUST_0003,Male,34,24,No,NaN,One year,87.21,2093.04,No
3,3,CUST_0004,Male,26,11,Yes,NaN,One year,50.39,554.29,No
4,4,CUST_0005,Male,50,14,Yes,DSL,Month-to-month,104.51,1463.14,Yes


In [ ]:
df = df.dropna()
df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors='coerce')
df = df.dropna(subset=["TotalCharges"])

df["Churn"] = df["Churn"].map({"Yes": 1, "No": 0})

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 660 entries, 0 to 999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       660 non-null    int64  
 1   CustomerID       660 non-null    object 
 2   Gender           660 non-null    object 
 3   Age              660 non-null    int64  
 4   Tenure           660 non-null    int64  
 5   PhoneService     660 non-null    object 
 6   InternetService  660 non-null    object 
 7   Contract         660 non-null    object 
 8   MonthlyCharges   660 non-null    float64
 9   TotalCharges     660 non-null    float64
 10  Churn            660 non-null    int64  
dtypes: float64(2), int64(4), object(5)
memory usage: 61.9+ KB


/tmp/ipython-input-27-3389393371.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors='coerce')


,Unnamed: 0,CustomerID,Gender,Age,Tenure,PhoneService,InternetService,Contract,MonthlyCharges,TotalCharges,Churn
0,0,CUST_0001,Male,71,69,Yes,Fiber optic,Month-to-month,45.08,3110.52,0
1,1,CUST_0002,Female,80,71,No,DSL,Month-to-month,80.37,5706.27,1
4,4,CUST_0005,Male,50,14,Yes,DSL,Month-to-month,104.51,1463.14,1
6,6,CUST_0007,Male,37,25,No,DSL,Month-to-month,108.26,2706.50,0
7,7,CUST_0008,Male,30,25,Yes,Fiber optic,One year,93.38,2334.50,0


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = df.drop("Churn", axis=1)
y = df["Churn"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Training:", X_train.shape, "Test:", X_test.shape)

Training: (528, 10) Test: (132, 10)


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier

In [ ]:
numeric_cols = X.select_dtypes(include='number').columns
categorical_cols = X.select_dtypes(exclude='number').columns

preprocessor = ColumnTransformer([
    ("num", StandardScaler(), numeric_cols),
    ("cat", OneHotEncoder(handle_unknown='ignore'), categorical_cols)
])

pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", RandomForestClassifier(random_state=42))
])

pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  Index(['Unnamed: 0', 'Age', 'Tenure', 'MonthlyCharges', 'TotalCharges'], dtype='object')),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Index(['CustomerID', 'Gender', 'PhoneService', 'InternetService', 'Contract'], dtype='object'))])),
                ('classifier', RandomForestClassifier(random_state=42))])

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
param_dist = {
    "classifier__n_estimators": [50, 100, 200],
    "classifier__max_depth": [5, 10, 15, None]
}

search = RandomizedSearchCV(pipeline, param_distributions=param_dist, n_iter=5, scoring="roc_auc", cv=3, random_state=42)
search.fit(X_train, y_train)

print("Best params:", search.best_params_)

Best params: {'classifier__n_estimators': 50, 'classifier__max_depth': 5}


In [ ]:
from sklearn.metrics import classification_report, roc_auc_score

In [ ]:
y_pred = search.predict(X_test)
y_proba = search.predict_proba(X_test)[:, 1]

print(classification_report(y_test, y_pred))
print("ROC AUC Score:", roc_auc_score(y_test, y_proba))

              precision    recall  f1-score   support

           0       0.83      1.00      0.90       109
           1       0.00      0.00      0.00        23

    accuracy                           0.83       132
   macro avg       0.41      0.50      0.45       132
weighted avg       0.68      0.83      0.75       132

ROC AUC Score: 0.8809333865177504


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
